In [73]:
import requests
import json
import yaml
import pandas as pd
import time
import datetime

#### Get top 10 routes by ridership

In [9]:
# call getroutes to get route list from Bus Tracker API
config = open('config.yaml', 'r')
data = yaml.load(config)
key = data.get('CTA_BUS_TRACKER')

In [15]:
url = f'http://ctabustracker.com/bustime/api/v2/getroutes?key={key}&format=json'
#routes = pd.read_json(url) # doesn't work

In [18]:
r = requests.get(url)
route_list = r.json()

In [31]:
route_names = [i.get('rt') for i in route_list['bustime-response']['routes']]

In [35]:
# Get ridership data
averages = {}

for route in route_names:
    url = f'https://data.cityofchicago.org/resource/dv3z-wsyd.json?route={route}'
    temp = pd.read_json(url)
    averages[route]=temp.rides.mean()

In [39]:
sorted_averages = sorted(averages.items(), key=lambda x: x[1])

In [44]:
top_ten = [i[0] for i in sorted_averages[-10:]]

In [53]:
route_str = str(top_ten).replace('[','').replace(']','').replace(' ','').replace("'","")

In [55]:
# get vehicle locations for top ten bus routes
url = f'http://ctabustracker.com/bustime/api/v2/getvehicles?key={key}&rt={route_str}&tmres=s&format=json'

In [57]:
r = requests.get(url)
bus_locations = r.json()

In [60]:
bus_locations['bustime-response']['vehicle']

[{'vid': '7911',
  'tmstmp': '20180923 16:01:01',
  'lat': '41.72100067138672',
  'lon': '-87.60877227783203',
  'hdg': '87',
  'pid': 5342,
  'rt': '3',
  'des': 'Michigan/Chicago',
  'pdist': 65,
  'dly': False,
  'tatripid': '10001379',
  'tablockid': '3 -705',
  'zone': ''},
 {'vid': '1270',
  'tmstmp': '20180923 16:01:10',
  'lat': '41.74068214779808',
  'lon': '-87.61463928222656',
  'hdg': '358',
  'pid': 5342,
  'rt': '3',
  'des': 'Michigan/Chicago',
  'pdist': 8635,
  'dly': False,
  'tatripid': '10001378',
  'tablockid': '3 -711',
  'zone': ''},
 {'vid': '7935',
  'tmstmp': '20180923 16:00:51',
  'lat': '41.76975355477169',
  'lon': '-87.615316982927',
  'hdg': '359',
  'pid': 5342,
  'rt': '3',
  'des': 'Michigan/Chicago',
  'pdist': 19268,
  'dly': False,
  'tatripid': '10001377',
  'tablockid': '3 -704',
  'zone': ''},
 {'vid': '1315',
  'tmstmp': '20180923 16:01:02',
  'lat': '41.7941945810787',
  'lon': '-87.61588287353516',
  'hdg': '0',
  'pid': 5342,
  'rt': '3',
  '

In [84]:
# Time for a while loop
counter = 0
df_locations_all = pd.DataFrame()
while counter < 30:
    print('Counter at' + str(counter))
    start_time = datetime.datetime.now()
    r = requests.get(url)
    bus_locations = r.json()
    end_time = datetime.datetime.now()
    df_locations_temp = pd.DataFrame.from_dict(bus_locations['bustime-response']['vehicle'])
    df_locations_temp['call_num'] = counter
    df_locations_temp['start_time'] = start_time
    df_locations_temp['end_time'] = end_time
    df_locations_all = pd.concat([df_locations_temp, df_locations_all])
    counter = counter + 1
    time.sleep(300)
    
df_locations_all.to_csv('locations_all_test.csv')

Counter at0
Counter at1
Counter at2
Counter at3
Counter at4
Counter at5
Counter at6
Counter at7
Counter at8
Counter at9
Counter at10
Counter at11
Counter at12
Counter at13
Counter at14
Counter at15
Counter at16
Counter at17
Counter at18
Counter at19
Counter at20
Counter at21
Counter at22
Counter at23
Counter at24
Counter at25
Counter at26
Counter at27
Counter at28
Counter at29


In [85]:
df_locations_all.rt.value_counts()

79     405
9      362
4      337
49     331
3      330
151    312
22     277
66     219
63     161
20     153
Name: rt, dtype: int64

In [86]:
df_locations_all.shape

(2887, 16)

In [92]:
df_locations_all.query("vid=='1400'").sort_values('call_num')

,des,dly,hdg,lat,lon,pdist,pid,rt,tablockid,tatripid,tmstmp,vid,zone,call_num,start_time,end_time
118,Union Station,False,69,41.8777961730957,-87.64056061535346,57469,8102,151,151 -502,10001275,20180923 17:41:53,1400,,0,2018-09-23 17:42:18.488003,2018-09-23 17:42:18.900612
114,Union Station,True,69,41.8777961730957,-87.64056061535346,57469,8102,151,151 -502,10001275,20180923 17:47:00,1400,,1,2018-09-23 17:47:18.920563,2018-09-23 17:47:19.322977
116,Union Station,False,68,41.8778076171875,-87.63969421386719,57614,8102,151,151 -502,10001275,20180923 17:52:05,1400,,2,2018-09-23 17:52:19.341926,2018-09-23 17:52:19.723118
102,Devon/Clark,False,39,41.87802505493164,-87.63941192626953,214,6414,151,151 -502,10001277,20180923 17:56:50,1400,,3,2018-09-23 17:57:19.751380,2018-09-23 17:57:20.148854
96,Devon/Clark,False,359,41.984439544677734,-87.65518951416016,48149,6414,151,151 -502,10001277,20180923 19:01:59,1400,,4,2018-09-23 19:02:12.722393,2018-09-23 19:02:13.080327
93,Devon/Clark,False,357,41.99611129760742,-87.65548706054688,52382,6414,151,151 -502,10001277,20180923 19:07:00,1400,,5,2018-09-23 19:07:13.104542,2018-09-23 19:07:13.444094
96,Devon/Clark,False,271,41.99810457229614,-87.6703233718872,57016,6414,151,151 -502,10001277,20180923 19:12:02,1400,,6,2018-09-23 19:12:13.468959,2018-09-23 19:12:13.809925
93,Devon/Clark,False,266,41.999855,-87.67097833333334,57700,6414,151,151 -502,10001277,20180923 19:17:04,1400,,7,2018-09-23 19:17:13.836111,2018-09-23 19:17:14.186075
94,Devon/Clark,False,266,41.999855,-87.67097833333334,57700,6414,151,151 -502,10001277,20180923 19:21:40,1400,,8,2018-09-23 19:22:14.208857,2018-09-23 19:22:14.532574
91,Devon/Clark,False,266,41.99979019165039,-87.6709213256836,57700,6414,151,151 -502,10001277,20180923 19:27:04,1400,,9,2018-09-23 19:27:14.557057,2018-09-23 19:27:14.894119
